In [ ]:
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso, JLD2
using _Data, _Math

In [ ]:
@pyimport numpy
@pyimport sklearn.decomposition as decomposition
@pyimport scipy.stats as stats
@pyimport matplotlib.colors as mpl_colors
@pyimport matplotlib.cm as cm 
@pyimport sklearn.cluster as cluster

In [ ]:
rc_params = PyDict(pyimport("matplotlib")["rcParams"]);
rc_params["font.sans-serif"] = ["Arial"];
rc_params["font.size"] = 5;
rc_params["lines.linewidth"] = 1;
rc_params["lines.markersize"] = 4;
rc_params["xtick.major.size"] = 2;
rc_params["ytick.major.size"] = 2;
rc_params["xtick.major.pad"] = 2;
rc_params["ytick.major.pad"] = 2;
# rc_params["axes.labelpad"] = 2;
rc_params["axes.spines.right"] = false
rc_params["axes.spines.top"] = false

cim(img::Matrix{UInt32}) = CairoImageSurface(img, Cairo.FORMAT_RGB24; flipxy = false) 
cim(img::Matrix{ARGB32}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{RGB24}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{UInt8}) = cim(Gray.(reinterpret(N0f8, img)))
cim(img::Array{UInt8,3}) = cim(RGB24.(reinterpret(N0f8, img[:,:,1]), reinterpret(N0f8, img[:,:,2]), reinterpret(N0f8, img[:,:,3])));downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);
downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);

In [ ]:
include("../../../functions/func_map.jl")
include("../../../functions/func_stat.jl")
include("../../../functions/func_data.jl")
include("../../../functions/func_plot.jl")

In [ ]:
data_info_dict = load("../figure_data_info.jld2")

data_info_all = []
data_info_tog = []
job_names = []
for key in keys(data_info_dict)
    append!(data_info_all, data_info_dict[key])
    append!(data_info_tog, [data_info_dict[key]])
    append!(job_names, [String(key)])
end

In [ ]:
analyzer = "chuyu"

In [ ]:
using Printf
function plot_mean_density(bins, y_tog; plot_color="k", shade_color="gray", line_color="k", plot_median = true)
    
    n_bins = length(bins)
    x = (bins[1:end-1] + bins[2:end])/2
    y = mean(y_tog)
    ymax = y + std(y_tog)
    ymin = y - std(y_tog);
    
    cumsum_y_mean = cumsum(mean(y_tog)*(maximum(bins) - minimum(bins))/(n_bins-1))

    x_median = numpy.round(numpy.interp(0.5, cumsum_y_mean, x), 2)
    x_14 = numpy.round(numpy.interp(0.25, cumsum_y_mean, x), 2)
    x_34 = numpy.round(numpy.interp(0.75, cumsum_y_mean, x), 2)
    
    
    plot(x, y, color= plot_color)
    fill_between(x, ymax, ymin, color=shade_color, alpha=0.3)
    if plot_median
        axvline(x_median, color=line_color, label= @sprintf("%.2f",x_median), linestyle="dashed")
        # axvline(x_median, color=line_color, label= "$x_median ($x_14 - $x_34)")
    end
    
    

end

In [ ]:
job_names

In [ ]:
job_names_ordered = [
 "corner_cue_rotation_whole"
 "landmark_removal"
 "boundary_morphing"
 "job_retangular_half_circle_rotation"

    
 "hedgehog_rotation_bottom_out"
 "hedgehog_landmark_removal"
 "hedgehog_circle"
 "corner_cue_circle"
    
]


In [ ]:
job_names_figure = [
 "Chamber rotation"
 "Landmark removal"
 "Wall morphing"
    
 "Wall rotation"
    
 "No change"
 "Landmark removal"
 "Wall morphing"
 "Wall morphing + \n Landmark removal"
]


# Compared with control, original maps

In [ ]:
fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/sfigure6/summary/original"
mkpath(fig_path)

In [ ]:
fig = figure(figsize=(5,1.5))
tick_locs = []
tick_labels = []
for (i_job, job_name) in enumerate(job_names_ordered)
data_info_all = data_info_dict[job_name]
job_name = job_names_figure[i_job]


n_bins = 31

trials_median_tog = []
earlylate_median_tog = []
    
trials_all_exp = []    
earlylate_all_exp = []    
    
for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results.h5")
    file = h5open(info_filename, "r")
    tel_place_cell_index_1 = HDF5.readmmap(file["tel_place_cell_index_1"])
    tel_place_cell_index_2 = HDF5.readmmap(file["tel_place_cell_index_2"])
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])

    close(file)
        
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late.h5")
    file = h5open(info_filename, "r")
    corr_earlylate_all = HDF5.readmmap(file["corr_earlylate_all"])
    corr_pv_earlylate_all = HDF5.readmmap(file["corr_pv_earlylate_all"])
    corr_pv_earlylate_all_normalized = HDF5.readmmap(file["corr_pv_earlylate_all_normalized"])
    peak_shift_early_late_mm = HDF5.readmmap(file["peak_shift_early_late_mm"])
    specificity_1_early = HDF5.readmmap(file["specificity_1_early"])
    specificity_1_late = HDF5.readmmap(file["specificity_1_late"])
    close(file)
        
    if job_name != "boundary_morphing"
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_original.h5")
    else
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_original.h5") 
    end
    file = h5open(info_filename, "r")
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    corr_pv_all = HDF5.readmmap(file["corr_pv_all"])
    corr_pv_all_normalized = HDF5.readmmap(file["corr_pv_all_normalized"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    specificity_1 = HDF5.readmmap(file["specificity_1"])
    specificity_2 = HDF5.readmmap(file["specificity_2"])
    close(file)
    
    
    append!(trials_median_tog, nanmedian(corr_trials_all[tel_place_cell_index]))
    append!(earlylate_median_tog, nanmedian(corr_earlylate_all[tel_place_cell_index]))
        
        
    append!(trials_all_exp, [corr_trials_all[tel_place_cell_index]])
    append!(earlylate_all_exp, [corr_earlylate_all[tel_place_cell_index]])
end
    


siginficance_all = []
plot_y_all = []
for i in 1:length(earlylate_median_tog)
    x = reduce(vcat, trials_all_exp[i])
    y = reduce(vcat, earlylate_all_exp[i])

    valid_neuron = isfinite.(x) .* isfinite.(y)
    _,p_value = stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "less")
    siginficance = "n.s."
    if p_value < 0.01
        siginficance = "*"
    end
    if p_value < 1e-3
        siginficance = "**"
    end
    if p_value < 1e-5
        siginficance = "***"
    end
    println(p_value)
    append!(siginficance_all, [siginficance])
        
    
    plot([i_job*2-1,i_job*2], [earlylate_median_tog[i], trials_median_tog[i]], color="gray")
    scatter([i_job*2-1], [earlylate_median_tog[i]], color="gray", alpha=0.7)
    scatter([i_job*2], [trials_median_tog[i]], color="r", alpha=0.7)
    append!(plot_y_all, trials_median_tog[i] - 0.04)


end
plot_y_all_sort = plot_y_all[sortperm(-plot_y_all)]
siginficance_all_sort = siginficance_all[sortperm(-plot_y_all)] 

for i in 2:length(plot_y_all_sort)
    if plot_y_all_sort[i-1] - plot_y_all_sort[i]  < 0.03
            plot_y_all_sort[i:end] .= plot_y_all_sort[i:end] .- 0.03
        end
    end
    
for i in 1:length(earlylate_median_tog) 
    text(i_job*2 + 0.1, plot_y_all_sort[i], siginficance_all_sort[i])
end
    

    
    append!(tick_locs, [i_job*2-0.5])
    append!(tick_labels, [])

end

xticks(tick_locs, tick_labels, rotation=90)
ylabel("median PF correlation")

fig.savefig(joinpath(fig_path, "PF_correlation_compare_control.pdf"), bbox_inches="tight",transparent = true);

In [ ]:
fig = figure(figsize=(7,2))
tick_locs = []
tick_labels = []
for (i_job, job_name) in enumerate(job_names_ordered)
data_info_all = data_info_dict[job_name]
job_name = job_names_figure[i_job]

n_bins = 31

trials_median_tog = []
earlylate_median_tog = []
    
trials_all_exp = []    
earlylate_all_exp = []   
    

for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results.h5")
    file = h5open(info_filename, "r")
    tel_place_cell_index_1 = HDF5.readmmap(file["tel_place_cell_index_1"])
    tel_place_cell_index_2 = HDF5.readmmap(file["tel_place_cell_index_2"])
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])
    close(file)
        
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late.h5")
    file = h5open(info_filename, "r")
    corr_earlylate_all = HDF5.readmmap(file["corr_earlylate_all"])
    pv_map_normalized_early_late = HDF5.readmmap(file["pv_map_dFF"])
    peak_shift_early_late_mm = HDF5.readmmap(file["peak_shift_early_late_mm"])
    specificity_1_early = HDF5.readmmap(file["specificity_1_early"])
    specificity_1_late = HDF5.readmmap(file["specificity_1_late"])
    close(file)
        
    if job_name != "boundary_morphing"
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_original.h5")
    else
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_original.h5") 
    end
    file = h5open(info_filename, "r")
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_dFF"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    specificity_1 = HDF5.readmmap(file["specificity_1"])
    specificity_2 = HDF5.readmmap(file["specificity_2"])
    close(file)
        
        
    valid_index = isfinite.(pv_map_normalized_early_late) .* isfinite.(pv_map_normalized)
    append!(trials_median_tog, nanmedian(pv_map_normalized[valid_index]))
    append!(earlylate_median_tog, nanmedian(pv_map_normalized_early_late[valid_index]))
    

    append!(trials_all_exp, [pv_map_normalized[valid_index]])
    append!(earlylate_all_exp, [pv_map_normalized_early_late[valid_index]])   
end



siginficance_all = []
plot_y_all = []
for i in 1:length(earlylate_median_tog)
    x = reduce(vcat, trials_all_exp[i])
    y = reduce(vcat, earlylate_all_exp[i])

    valid_neuron = isfinite.(x) .* isfinite.(y)
    _,p_value = stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "less")
    siginficance = "n.s."
    if p_value < 0.01
        siginficance = "*"
    end
    if p_value < 1e-3
        siginficance = "**"
    end
    if p_value < 1e-5
        siginficance = "***"
    end
    println(p_value)
    append!(siginficance_all, [siginficance])
        
    
    plot([i_job*2-1,i_job*2], [earlylate_median_tog[i], trials_median_tog[i]], color="gray")
    scatter([i_job*2-1], [earlylate_median_tog[i]], color="gray", alpha=0.7)
    scatter([i_job*2], [trials_median_tog[i]], color="r", alpha=0.7)
    append!(plot_y_all, trials_median_tog[i] - 0.02)


end
plot_y_all_sort = plot_y_all[sortperm(-plot_y_all)]
siginficance_all_sort = siginficance_all[sortperm(-plot_y_all)] 

for i in 2:length(plot_y_all_sort)
    if plot_y_all_sort[i-1] - plot_y_all_sort[i]  < 0.02
            plot_y_all_sort[i:end] .= plot_y_all_sort[i:end] .- 0.02
        end
    end
    
for i in 1:length(earlylate_median_tog) 
    text(i_job*2 + 0.1, plot_y_all_sort[i], siginficance_all_sort[i])
end
    

    
    
    append!(tick_locs, [i_job*2-0.5])
    append!(tick_labels, [])

end

xticks(tick_locs, tick_labels, rotation=90)
ylabel("median PV correlation")
fig.savefig(joinpath(fig_path, "PV_correlation_compare_control.pdf"), bbox_inches="tight",transparent = true);

In [ ]:
fig = figure(figsize=(7,2))
tick_locs = []
tick_labels = []
for (i_job, job_name) in enumerate(job_names_ordered)
data_info_all = data_info_dict[job_name]
job_name = job_names_figure[i_job]

n_bins = 31

trials_median_tog = []
earlylate_median_tog = []
    
trials_all_exp = []    
earlylate_all_exp = []  
    

for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    
    if job_name != "boundary_morphing"
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_original.h5")
    else
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_original.h5") 
    end
    file = h5open(info_filename, "r")
    tel_place_cell_index_1 = HDF5.readmmap(file["tel_place_cell_index_1"])
    tel_place_cell_index_2 = HDF5.readmmap(file["tel_place_cell_index_2"])
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    corr_pv_all = HDF5.readmmap(file["corr_pv_all"])
    corr_pv_all_normalized = HDF5.readmmap(file["corr_pv_all_normalized"])
    pv_map = HDF5.readmmap(file["pv_map"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_normalized"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    specificity_1 = HDF5.readmmap(file["specificity_1"])
    specificity_2 = HDF5.readmmap(file["specificity_2"])
    close(file)
        
        
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late.h5")
    file = h5open(info_filename, "r")

    corr_earlylate_all = HDF5.readmmap(file["corr_earlylate_all"])
    corr_pv_earlylate_all = HDF5.readmmap(file["corr_pv_earlylate_all"])
    corr_pv_earlylate_all_normalized = HDF5.readmmap(file["corr_pv_earlylate_all_normalized"])
    peak_shift_early_late_mm = HDF5.readmmap(file["peak_shift_early_late_mm"])
    specificity_1_early = HDF5.readmmap(file["specificity_1_early"])
    specificity_1_late = HDF5.readmmap(file["specificity_1_late"])
    pv_map_earlylate = HDF5.readmmap(file["pv_map"])
    pv_map_normalized_earlylate = HDF5.readmmap(file["pv_map_normalized"])
    confined_place_cell_index_early_late = HDF5.readmmap(file["confined_place_cell_index"])
    close(file)
        
        
    append!(trials_median_tog, nanmedian(peak_shift_mm[whether_in(confined_place_cell_index,confined_place_cell_index_early_late)]))
    append!(earlylate_median_tog, nanmedian(peak_shift_early_late_mm[whether_in(confined_place_cell_index_early_late, confined_place_cell_index)]))
    
    append!(trials_all_exp, [peak_shift_mm[whether_in(confined_place_cell_index,confined_place_cell_index_early_late)]])
    append!(earlylate_all_exp, [peak_shift_early_late_mm[whether_in(confined_place_cell_index_early_late, confined_place_cell_index)]])   
    
    
end


siginficance_all = []
plot_y_all = []
for i in 1:length(earlylate_median_tog)
    x = reduce(vcat, trials_all_exp[i])
    y = reduce(vcat, earlylate_all_exp[i])

    valid_neuron = isfinite.(x) .* isfinite.(y)
    _,p_value = stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "greater")
    siginficance = "n.s."
    if p_value < 0.01
        siginficance = "*"
    end
    if p_value < 1e-3
        siginficance = "**"
    end
    if p_value < 1e-5
        siginficance = "***"
    end
    println(p_value)
    append!(siginficance_all, [siginficance])
        
    
    plot([i_job*2-1,i_job*2], [earlylate_median_tog[i], trials_median_tog[i]], color="gray")
    scatter([i_job*2-1], [earlylate_median_tog[i]], color="gray", alpha=0.7)
    scatter([i_job*2], [trials_median_tog[i]], color="r", alpha=0.7)
    append!(plot_y_all, trials_median_tog[i] - 0.04)


end
plot_y_all_sort = plot_y_all[sortperm(-plot_y_all)]
siginficance_all_sort = siginficance_all[sortperm(-plot_y_all)] 

for i in 2:length(plot_y_all_sort)
    if plot_y_all_sort[i-1] - plot_y_all_sort[i]  < 2
            plot_y_all_sort[i:end] .= plot_y_all_sort[i:end] .- 2
        end
    end
    
for i in 1:length(earlylate_median_tog) 
    text(i_job*2 + 0.1, plot_y_all_sort[i], siginficance_all_sort[i])
end
    

    
    
    append!(tick_locs, [i_job*2-0.5])
    append!(tick_labels, [])

end

xticks(tick_locs, tick_labels, rotation=45)
ylabel("median Peak shift (mm)")

ylim(0,45)
fig.savefig(joinpath(fig_path, "peak_shift_compare_control.pdf"), bbox_inches="tight",transparent = true);

# Match angle

In [ ]:
fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/sfigure6/summary/match_angle"
mkpath(fig_path)

In [ ]:
fig = figure(figsize=(7,1.5))
tick_locs = []
tick_labels = []
for (i_job, job_name) in enumerate(job_names_ordered)
data_info_all = data_info_dict[job_name]
job_name = job_names_figure[i_job]


n_bins = 31

trials_median_tog = []
earlylate_median_tog = []
    

for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results.h5")
    file = h5open(info_filename, "r")
    tel_place_cell_index_1 = HDF5.readmmap(file["tel_place_cell_index_1"])
    tel_place_cell_index_2 = HDF5.readmmap(file["tel_place_cell_index_2"])
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])

    close(file)
        
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late.h5")
    file = h5open(info_filename, "r")
    corr_earlylate_all = HDF5.readmmap(file["corr_earlylate_all"])
    corr_pv_earlylate_all = HDF5.readmmap(file["corr_pv_earlylate_all"])
    corr_pv_earlylate_all_normalized = HDF5.readmmap(file["corr_pv_earlylate_all_normalized"])
    peak_shift_early_late_mm = HDF5.readmmap(file["peak_shift_early_late_mm"])
    specificity_1_early = HDF5.readmmap(file["specificity_1_early"])
    specificity_1_late = HDF5.readmmap(file["specificity_1_late"])
    close(file)
        
    if job_name != "boundary_morphing"
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchangle.h5")
    else
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchboundary.h5") 
    end
    file = h5open(info_filename, "r")
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    corr_pv_all = HDF5.readmmap(file["corr_pv_all"])
    corr_pv_all_normalized = HDF5.readmmap(file["corr_pv_all_normalized"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    specificity_1 = HDF5.readmmap(file["specificity_1"])
    specificity_2 = HDF5.readmmap(file["specificity_2"])
    close(file)
    
    
    append!(trials_median_tog, nanmedian(corr_trials_all[tel_place_cell_index]))
    append!(earlylate_median_tog, nanmedian(corr_earlylate_all[tel_place_cell_index]))
    

    
    
end



for i in 1:length(earlylate_median_tog)
    plot([i_job*2-1,i_job*2], [earlylate_median_tog[i], trials_median_tog[i]], color="gray")
    scatter([i_job*2-1], [earlylate_median_tog[i]], color="gray", alpha=0.7)
    scatter([i_job*2], [trials_median_tog[i]], color="r", alpha=0.7)

end

    
    append!(tick_locs, [i_job*2-0.5])
    append!(tick_labels, [])

end

xticks(tick_locs, tick_labels, rotation=90)
ylabel("median PF correlation")

fig.savefig(joinpath(fig_path, "PF_correlation_compare_control.pdf"), bbox_inches="tight",transparent = true);

In [ ]:
fig = figure(figsize=(7,1.5))
tick_locs = []
tick_labels = []
for (i_job, job_name) in enumerate(job_names_ordered)
data_info_all = data_info_dict[job_name]
job_name = job_names_figure[i_job]

n_bins = 31

trials_median_tog = []
earlylate_median_tog = []
    

for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results.h5")
    file = h5open(info_filename, "r")
    tel_place_cell_index_1 = HDF5.readmmap(file["tel_place_cell_index_1"])
    tel_place_cell_index_2 = HDF5.readmmap(file["tel_place_cell_index_2"])
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])
    close(file)
        
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late.h5")
    file = h5open(info_filename, "r")
    corr_earlylate_all = HDF5.readmmap(file["corr_earlylate_all"])
    pv_map_normalized_early_late = HDF5.readmmap(file["pv_map_dFF"])
    peak_shift_early_late_mm = HDF5.readmmap(file["peak_shift_early_late_mm"])
    specificity_1_early = HDF5.readmmap(file["specificity_1_early"])
    specificity_1_late = HDF5.readmmap(file["specificity_1_late"])
    close(file)
        
    if job_name != "boundary_morphing"
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchangle.h5")
    else
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchboundary.h5") 
    end
    file = h5open(info_filename, "r")
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_dFF"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    specificity_1 = HDF5.readmmap(file["specificity_1"])
    specificity_2 = HDF5.readmmap(file["specificity_2"])
    close(file)
        
        
    valid_index = isfinite.(pv_map_normalized_early_late) .* isfinite.(pv_map_normalized)
    append!(trials_median_tog, nanmedian(pv_map_normalized[valid_index]))
    append!(earlylate_median_tog, nanmedian(pv_map_normalized_early_late[valid_index]))
    

    
    
end



for i in 1:length(earlylate_median_tog)
    plot([i_job*2-1,i_job*2], [earlylate_median_tog[i], trials_median_tog[i]], color="gray")
    scatter([i_job*2-1], [earlylate_median_tog[i]], color="gray", alpha=0.7)
    scatter([i_job*2], [trials_median_tog[i]], color="r", alpha=0.7)

end

    
    
    append!(tick_locs, [i_job*2-0.5])
    append!(tick_labels, [])

end

xticks(tick_locs, tick_labels, rotation=90)
ylabel("median PV correlation")
fig.savefig(joinpath(fig_path, "PV_correlation_compare_control.pdf"), bbox_inches="tight",transparent = true);

In [ ]:
fig = figure(figsize=(7,1))
tick_locs = []
tick_labels = []
for (i_job, job_name) in enumerate(job_names_ordered)
data_info_all = data_info_dict[job_name]
job_name = job_names_figure[i_job]

n_bins = 31

trials_median_tog = []
earlylate_median_tog = []
    

for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    
    if job_name != "boundary_morphing"
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchangle.h5")
    else
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchboundary.h5") 
    end
    file = h5open(info_filename, "r")
    tel_place_cell_index_1 = HDF5.readmmap(file["tel_place_cell_index_1"])
    tel_place_cell_index_2 = HDF5.readmmap(file["tel_place_cell_index_2"])
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    corr_pv_all = HDF5.readmmap(file["corr_pv_all"])
    corr_pv_all_normalized = HDF5.readmmap(file["corr_pv_all_normalized"])
    pv_map = HDF5.readmmap(file["pv_map"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_normalized"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    specificity_1 = HDF5.readmmap(file["specificity_1"])
    specificity_2 = HDF5.readmmap(file["specificity_2"])
    close(file)
        
        
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late.h5")
    file = h5open(info_filename, "r")

    corr_earlylate_all = HDF5.readmmap(file["corr_earlylate_all"])
    corr_pv_earlylate_all = HDF5.readmmap(file["corr_pv_earlylate_all"])
    corr_pv_earlylate_all_normalized = HDF5.readmmap(file["corr_pv_earlylate_all_normalized"])
    peak_shift_early_late_mm = HDF5.readmmap(file["peak_shift_early_late_mm"])
    specificity_1_early = HDF5.readmmap(file["specificity_1_early"])
    specificity_1_late = HDF5.readmmap(file["specificity_1_late"])
    pv_map_earlylate = HDF5.readmmap(file["pv_map"])
    pv_map_normalized_earlylate = HDF5.readmmap(file["pv_map_normalized"])
    confined_place_cell_index_early_late = HDF5.readmmap(file["confined_place_cell_index"])
    close(file)
        
        
    append!(trials_median_tog, nanmedian(peak_shift_mm[whether_in(confined_place_cell_index,confined_place_cell_index_early_late)]))
    append!(earlylate_median_tog, nanmedian(peak_shift_early_late_mm[whether_in(confined_place_cell_index_early_late, confined_place_cell_index)]))
    

    
    
end



for i in 1:length(earlylate_median_tog)
    plot([i_job*2-1,i_job*2], [earlylate_median_tog[i], trials_median_tog[i]], color="gray")
    scatter([i_job*2-1], [earlylate_median_tog[i]], color="gray", alpha=0.7)
    scatter([i_job*2], [trials_median_tog[i]], color="r", alpha=0.7)

end

    
    
    append!(tick_locs, [i_job*2-0.5])
    append!(tick_labels, [job_name])

end

xticks(tick_locs, tick_labels, rotation=45)
ylabel("median Peak shift (mm)")

ylim(0,45)
fig.savefig(joinpath(fig_path, "peak_shift_compare_control.pdf"), bbox_inches="tight",transparent = true);

# Best morphed maps

In [ ]:
fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/sfigure6/summary/best"
mkpath(fig_path)

In [ ]:
fig = figure(figsize=(4.5,0.6))
tick_locs = []
tick_labels = []
for (i_job, job_name) in enumerate(job_names_ordered)
data_info_all = data_info_dict[job_name]
job_name = job_names_figure[i_job]


n_bins = 31

trials_median_tog = []
earlylate_median_tog = []
    
trials_all_exp = []    
earlylate_all_exp = []
    

for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results.h5")
    file = h5open(info_filename, "r")
    tel_place_cell_index_1 = HDF5.readmmap(file["tel_place_cell_index_1"])
    tel_place_cell_index_2 = HDF5.readmmap(file["tel_place_cell_index_2"])
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index_1 = HDF5.readmmap(file["confined_place_cell_index_1"])
    confined_place_cell_index_2 = HDF5.readmmap(file["confined_place_cell_index_2"])
    confined_place_cell_index = intersect(confined_place_cell_index_1, confined_place_cell_index_2)

    close(file)
        
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late_best.h5")
    file = h5open(info_filename, "r")
    corr_earlylate_all = HDF5.readmmap(file["corr_earlylate_all"])
    corr_pv_earlylate_all = HDF5.readmmap(file["corr_pv_earlylate_all"])
    corr_pv_earlylate_all_normalized = HDF5.readmmap(file["corr_pv_earlylate_all_normalized"])
    peak_shift_early_late_mm = HDF5.readmmap(file["peak_shift_early_late_mm"])
    specificity_1_early = HDF5.readmmap(file["specificity_1_early"])
    specificity_1_late = HDF5.readmmap(file["specificity_1_late"])
    close(file)
        
    if job_name != "boundary_morphing"
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results.h5")
    else
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results.h5") 
    end
    file = h5open(info_filename, "r")
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    corr_pv_all = HDF5.readmmap(file["corr_pv_all"])
    corr_pv_all_normalized = HDF5.readmmap(file["corr_pv_all_normalized"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    specificity_1 = HDF5.readmmap(file["specificity_1"])
    specificity_2 = HDF5.readmmap(file["specificity_2"])
    close(file)
    
    
    append!(trials_median_tog, nanmedian(corr_trials_all[tel_place_cell_index]))
    append!(earlylate_median_tog, nanmedian(corr_earlylate_all[tel_place_cell_index]))
    
    append!(trials_all_exp, [corr_trials_all[tel_place_cell_index]])
    append!(earlylate_all_exp, [corr_earlylate_all[tel_place_cell_index]])


    
    
end

i_plot = i_job
if i_job>4
        i_plot = i_job+1
    end

siginficance_all = []
plot_y_all = []
for i in 1:length(earlylate_median_tog)
    x = reduce(vcat, trials_all_exp[i])
    y = reduce(vcat, earlylate_all_exp[i])

    valid_neuron = isfinite.(x) .* isfinite.(y)
    _,p_value = stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "less")
    siginficance = "n.s."
    if p_value < 0.01
        siginficance = "*"
    end
    if p_value < 1e-3
        siginficance = "**"
    end
    if p_value < 1e-5
        siginficance = "***"
    end
    println(p_value)
    append!(siginficance_all, [siginficance])
        
    
    plot([i_plot*2-1,i_plot*2], [earlylate_median_tog[i], trials_median_tog[i]], color="gray")
    scatter([i_plot*2-1], [earlylate_median_tog[i]], color="gray", alpha=0.7, s = 5)
    scatter([i_plot*2], [trials_median_tog[i]], color="r", alpha=0.7, s = 5)
    append!(plot_y_all, trials_median_tog[i])


end
plot_y_all_sort = plot_y_all[sortperm(-plot_y_all)]
siginficance_all_sort = siginficance_all[sortperm(-plot_y_all)] 

for i in 2:length(plot_y_all_sort)
    if plot_y_all_sort[i-1] - plot_y_all_sort[i]  < 0.07
            plot_y_all_sort[i] = plot_y_all_sort[i-1] -  0.07
        end
    end
    
for i in 1:length(earlylate_median_tog) 
    text(i_plot*2 + 0.15, plot_y_all_sort[i], siginficance_all_sort[i])
end

    
    append!(tick_locs, [i_plot*2-0.5])
    append!(tick_labels, [])

end

xticks(tick_locs, tick_labels, rotation=90)
ylabel("PF correlation")
ylim(-0,0.8)
fig.savefig(joinpath(fig_path, "PF_correlation_compare_control.pdf"), bbox_inches="tight",transparent = true);

In [ ]:
fig = figure(figsize=(4.5,0.6))
tick_locs = []
tick_labels = []
for (i_job, job_name) in enumerate(job_names_ordered)
data_info_all = data_info_dict[job_name]
job_name = job_names_figure[i_job]

n_bins = 31

trials_median_tog = []
earlylate_median_tog = []
    
trials_all_exp = []    
earlylate_all_exp = []
    

for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results.h5")
    file = h5open(info_filename, "r")
    tel_place_cell_index_1 = HDF5.readmmap(file["tel_place_cell_index_1"])
    tel_place_cell_index_2 = HDF5.readmmap(file["tel_place_cell_index_2"])
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index_1 = HDF5.readmmap(file["confined_place_cell_index_1"])
    confined_place_cell_index_2 = HDF5.readmmap(file["confined_place_cell_index_2"])
    confined_place_cell_index = intersect(confined_place_cell_index_1, confined_place_cell_index_2)
    close(file)
        
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late_best.h5")
    file = h5open(info_filename, "r")
    corr_earlylate_all = HDF5.readmmap(file["corr_earlylate_all"])
    pv_map_normalized_early_late = HDF5.readmmap(file["pv_map_dFF"])
    peak_shift_early_late_mm = HDF5.readmmap(file["peak_shift_early_late_mm"])
    specificity_1_early = HDF5.readmmap(file["specificity_1_early"])
    specificity_1_late = HDF5.readmmap(file["specificity_1_late"])
    close(file)
        
    if job_name != "boundary_morphing"
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results.h5")
    else
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results.h5") 
    end
    file = h5open(info_filename, "r")
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_dFF"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    specificity_1 = HDF5.readmmap(file["specificity_1"])
    specificity_2 = HDF5.readmmap(file["specificity_2"])
    close(file)
        
        
    valid_index = isfinite.(pv_map_normalized_early_late) .* isfinite.(pv_map_normalized)
    append!(trials_median_tog, nanmedian(pv_map_normalized[valid_index]))
    append!(earlylate_median_tog, nanmedian(pv_map_normalized_early_late[valid_index]))
    

    append!(trials_all_exp, [pv_map_normalized[valid_index]])
    append!(earlylate_all_exp, [pv_map_normalized_early_late[valid_index]])   


    
    
end


i_plot = i_job
if i_job>4
        i_plot = i_job+1
    end


siginficance_all = []
plot_y_all = []
for i in 1:length(earlylate_median_tog)
    x = reduce(vcat, trials_all_exp[i])
    y = reduce(vcat, earlylate_all_exp[i])

    valid_neuron = isfinite.(x) .* isfinite.(y)
    _,p_value = stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "less")
    siginficance = "n.s."
    if p_value < 0.01
        siginficance = "*"
    end
    if p_value < 1e-3
        siginficance = "**"
    end
    if p_value < 1e-5
        siginficance = "***"
    end
    println(p_value)
    append!(siginficance_all, [siginficance])
        
    
    plot([i_plot*2-1,i_plot*2], [earlylate_median_tog[i], trials_median_tog[i]], color="gray")
    scatter([i_plot*2-1], [earlylate_median_tog[i]], color="gray", alpha=0.7, s = 5)
    scatter([i_plot*2], [trials_median_tog[i]], color="r", alpha=0.7, s = 5)
    append!(plot_y_all, trials_median_tog[i])


end
plot_y_all_sort = plot_y_all[sortperm(-plot_y_all)]
siginficance_all_sort = siginficance_all[sortperm(-plot_y_all)] 

for i in 2:length(plot_y_all_sort)
    if plot_y_all_sort[i-1] - plot_y_all_sort[i]  < 0.07
            plot_y_all_sort[i] = plot_y_all_sort[i-1] -  0.07
        end
    end
    
for i in 1:length(earlylate_median_tog) 
    text(i_plot*2 + 0.15, plot_y_all_sort[i], siginficance_all_sort[i])
end

    
    
    append!(tick_locs, [i_plot*2-0.5])
    append!(tick_labels, [])

end

yticks([0, 0.2, 0.4, 0.6])

xticks(tick_locs, tick_labels, rotation=90)
ylabel("PV correlation")
fig.savefig(joinpath(fig_path, "PV_correlation_compare_control.pdf"), bbox_inches="tight",transparent = true);

In [ ]:
fig = figure(figsize=(4.5,0.6))
tick_locs = []
tick_labels = []
for (i_job, job_name) in enumerate(job_names_ordered)
data_info_all = data_info_dict[job_name]
job_name = job_names_figure[i_job]

n_bins = 31

trials_median_tog = []
earlylate_median_tog = []
    
trials_all_exp = []    
earlylate_all_exp = []  
    

for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    
    if job_name != "boundary_morphing"
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results.h5")
    else
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results.h5") 
    end
    file = h5open(info_filename, "r")
    tel_place_cell_index_1 = HDF5.readmmap(file["tel_place_cell_index_1"])
    tel_place_cell_index_2 = HDF5.readmmap(file["tel_place_cell_index_2"])
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index_1 = HDF5.readmmap(file["confined_place_cell_index_1"])
    confined_place_cell_index_2 = HDF5.readmmap(file["confined_place_cell_index_2"])
    confined_place_cell_index = intersect(confined_place_cell_index_1, confined_place_cell_index_2)
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    corr_pv_all = HDF5.readmmap(file["corr_pv_all"])
    corr_pv_all_normalized = HDF5.readmmap(file["corr_pv_all_normalized"])
    pv_map = HDF5.readmmap(file["pv_map"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_normalized"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    specificity_1 = HDF5.readmmap(file["specificity_1"])
    specificity_2 = HDF5.readmmap(file["specificity_2"])
    close(file)
        
        
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late_best.h5")
    file = h5open(info_filename, "r")

    corr_earlylate_all = HDF5.readmmap(file["corr_earlylate_all"])
    corr_pv_earlylate_all = HDF5.readmmap(file["corr_pv_earlylate_all"])
    corr_pv_earlylate_all_normalized = HDF5.readmmap(file["corr_pv_earlylate_all_normalized"])
    peak_shift_early_late_mm = HDF5.readmmap(file["peak_shift_early_late_mm"])
    specificity_1_early = HDF5.readmmap(file["specificity_1_early"])
    specificity_1_late = HDF5.readmmap(file["specificity_1_late"])
    pv_map_earlylate = HDF5.readmmap(file["pv_map"])
    pv_map_normalized_earlylate = HDF5.readmmap(file["pv_map_normalized"])
    confined_place_cell_index_early_late = HDF5.readmmap(file["confined_place_cell_index"])
    close(file)
        
        
    append!(trials_median_tog, nanmedian(peak_shift_mm[whether_in(confined_place_cell_index,confined_place_cell_index_early_late)]))
    append!(earlylate_median_tog, nanmedian(peak_shift_early_late_mm[whether_in(confined_place_cell_index_early_late, confined_place_cell_index)]))
    
    append!(trials_all_exp, [peak_shift_mm[whether_in(confined_place_cell_index,confined_place_cell_index_early_late)]])
    append!(earlylate_all_exp, [peak_shift_early_late_mm[whether_in(confined_place_cell_index_early_late, confined_place_cell_index)]])   
    

    
    
end



i_plot = i_job
if i_job>4
        i_plot = i_job+1
    end

siginficance_all = []
plot_y_all = []
for i in 1:length(earlylate_median_tog)
    x = reduce(vcat, trials_all_exp[i])
    y = reduce(vcat, earlylate_all_exp[i])

    valid_neuron = isfinite.(x) .* isfinite.(y)
    _,p_value = stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "greater")
    siginficance = "n.s."
    if p_value < 0.01
        siginficance = "*"
    end
    if p_value < 1e-3
        siginficance = "**"
    end
    if p_value < 1e-5
        siginficance = "***"
    end
    println(p_value)
    append!(siginficance_all, [siginficance])
        
    
    plot([i_plot*2-1,i_plot*2], [earlylate_median_tog[i], trials_median_tog[i]], color="gray")
    scatter([i_plot*2-1], [earlylate_median_tog[i]], color="gray", alpha=0.7, s = 5)
    scatter([i_plot*2], [trials_median_tog[i]], color="r", alpha=0.7, s = 5)
    append!(plot_y_all, trials_median_tog[i])


end
plot_y_all_sort = plot_y_all[sortperm(-plot_y_all)]
siginficance_all_sort = siginficance_all[sortperm(-plot_y_all)] 

for i in 2:length(plot_y_all_sort)
    if plot_y_all_sort[i-1] - plot_y_all_sort[i]  < 2
            plot_y_all_sort[i] = plot_y_all_sort[i-1] - 2
        end
    end
    
for i in 1:length(earlylate_median_tog) 
    text(i_plot*2 + 0.15, plot_y_all_sort[i], siginficance_all_sort[i])
end


    
    
    append!(tick_locs, [i_plot*2-0.5])
    append!(tick_labels, [job_name])

end

xticks(tick_locs, tick_labels, rotation=45, ha="right" , rotation_mode="anchor")
ylabel("PF shift (mm)")

ylim(-1,20)
fig.savefig(joinpath(fig_path, "peak_shift_compare_control.pdf"), bbox_inches="tight",transparent = true);

# Original + match angle (for wall rotations)

In [ ]:
fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/sfigure6/summary/"
mkpath(fig_path)

In [ ]:
job_names_figure

In [ ]:
fig = figure(figsize=(7,1.5))
tick_locs = []
tick_labels = []
i_plot = 1
for (i_job, job_name_original) in enumerate(job_names_ordered)
data_info_all = data_info_dict[job_name_original]
job_name = job_names_figure[i_job]

n_bins = 31

trials_median_tog = []
earlylate_median_tog = []
    
trials_all_exp = []    
earlylate_all_exp = []
    
for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    
    if job_name_original == "boundary_morphing" 
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchangle.h5") 
    elseif job_name_original == "corner_cue_rotation_whole" || job_name_original == "job_retangular_half_circle_rotation" 
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_original.h5")
    else
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchangle.h5")
    end
    file = h5open(info_filename, "r")
    tel_place_cell_index_1 = HDF5.readmmap(file["tel_place_cell_index_1"])
    tel_place_cell_index_2 = HDF5.readmmap(file["tel_place_cell_index_2"])
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    corr_pv_all = HDF5.readmmap(file["corr_pv_all"])
    corr_pv_all_normalized = HDF5.readmmap(file["corr_pv_all_normalized"])
    pv_map = HDF5.readmmap(file["pv_map"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_normalized"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    specificity_1 = HDF5.readmmap(file["specificity_1"])
    specificity_2 = HDF5.readmmap(file["specificity_2"])
    close(file)
        
        
    if info_filename == joinpath(data_path(ds_save_cy_1), "compare_map_results_original.h5")
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late_original.h5")
    else
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late.h5")
    end
        
    file = h5open(info_filename, "r")

    corr_earlylate_all = HDF5.readmmap(file["corr_earlylate_all"])
    corr_pv_earlylate_all = HDF5.readmmap(file["corr_pv_earlylate_all"])
    corr_pv_earlylate_all_normalized = HDF5.readmmap(file["corr_pv_earlylate_all_normalized"])
    peak_shift_early_late_mm = HDF5.readmmap(file["peak_shift_early_late_mm"])
    specificity_1_early = HDF5.readmmap(file["specificity_1_early"])
    specificity_1_late = HDF5.readmmap(file["specificity_1_late"])
    pv_map_earlylate = HDF5.readmmap(file["pv_map"])
    pv_map_normalized_earlylate = HDF5.readmmap(file["pv_map_normalized"])
    confined_place_cell_index_early_late = HDF5.readmmap(file["confined_place_cell_index"])
    close(file)
        
        
    append!(trials_median_tog, nanmedian(corr_trials_all[tel_place_cell_index]))
    append!(earlylate_median_tog, nanmedian(corr_earlylate_all[tel_place_cell_index][isfinite.(corr_trials_all[tel_place_cell_index])]))
    
    append!(trials_all_exp, [corr_trials_all[tel_place_cell_index]])
    append!(earlylate_all_exp, [corr_earlylate_all[tel_place_cell_index]])

    
    
end



        

siginficance_all = []
plot_y_all = []
for i in 1:length(earlylate_median_tog)
    x = reduce(vcat, trials_all_exp[i])
    y = reduce(vcat, earlylate_all_exp[i])

    valid_neuron = isfinite.(x) .* isfinite.(y)
    _,p_value = stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "less")
    siginficance = "n.s."
    if p_value < 0.01
        siginficance = "*"
    end
    if p_value < 1e-3
        siginficance = "**"
    end
    if p_value < 1e-5
        siginficance = "***"
    end
    println(p_value)
    append!(siginficance_all, [siginficance])
        
    
    plot([i_plot*2-1,i_plot*2], [earlylate_median_tog[i], trials_median_tog[i]], color="gray")
    scatter([i_plot*2-1], [earlylate_median_tog[i]], color="gray", alpha=0.7)
    scatter([i_plot*2], [trials_median_tog[i]], color="r", alpha=0.7)
    append!(plot_y_all, trials_median_tog[i])


end
plot_y_all_sort = plot_y_all[sortperm(-plot_y_all)]
siginficance_all_sort = siginficance_all[sortperm(-plot_y_all)] 

for i in 2:length(plot_y_all_sort)
    if plot_y_all_sort[i-1] - plot_y_all_sort[i]  < 0.07
            plot_y_all_sort[i] = plot_y_all_sort[i-1] -  0.07
        end
    end
    
for i in 1:length(earlylate_median_tog) 
    text(i_plot*2 + 0.15, plot_y_all_sort[i], siginficance_all_sort[i])
end


    
    append!(tick_locs, [i_plot*2-0.5])
    append!(tick_labels, [])
if job_name == "Chamber rotation"
    i_plot +=1
end
    
if job_name == "Wall rotation"
    i_plot +=2
end
    i_plot+=1
    
println(i_plot)
end





already_plot = 0
for (i_job, job_name_original) in enumerate(job_names_ordered)
data_info_all = data_info_dict[job_name_original]
job_name = job_names_figure[i_job]
    
if job_name != "Chamber rotation" && job_name != "Wall rotation"
    continue
end
    

n_bins = 31

trials_median_tog = []
earlylate_median_tog = []
    
trials_all_exp = []    
earlylate_all_exp = []  
    

for which_data in 1:length(data_info_all)
        
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    
    if job_name_original != "boundary_morphing"
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchangle.h5")
    else
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchboundary.h5") 
    end
    file = h5open(info_filename, "r")
    tel_place_cell_index_1 = HDF5.readmmap(file["tel_place_cell_index_1"])
    tel_place_cell_index_2 = HDF5.readmmap(file["tel_place_cell_index_2"])
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    corr_pv_all = HDF5.readmmap(file["corr_pv_all"])
    corr_pv_all_normalized = HDF5.readmmap(file["corr_pv_all_normalized"])
    pv_map = HDF5.readmmap(file["pv_map"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_normalized"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    specificity_1 = HDF5.readmmap(file["specificity_1"])
    specificity_2 = HDF5.readmmap(file["specificity_2"])
    close(file)
        
        
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late.h5")
    file = h5open(info_filename, "r")

    corr_earlylate_all = HDF5.readmmap(file["corr_earlylate_all"])
    corr_pv_earlylate_all = HDF5.readmmap(file["corr_pv_earlylate_all"])
    corr_pv_earlylate_all_normalized = HDF5.readmmap(file["corr_pv_earlylate_all_normalized"])
    peak_shift_early_late_mm = HDF5.readmmap(file["peak_shift_early_late_mm"])
    specificity_1_early = HDF5.readmmap(file["specificity_1_early"])
    specificity_1_late = HDF5.readmmap(file["specificity_1_late"])
    pv_map_earlylate = HDF5.readmmap(file["pv_map"])
    pv_map_normalized_earlylate = HDF5.readmmap(file["pv_map_normalized"])
    confined_place_cell_index_early_late = HDF5.readmmap(file["confined_place_cell_index"])
    close(file)
        
        
    append!(trials_median_tog, nanmedian(corr_trials_all[tel_place_cell_index]))
    append!(earlylate_median_tog, nanmedian(corr_earlylate_all[tel_place_cell_index][isfinite.(corr_trials_all[tel_place_cell_index])]))
    
   append!(trials_all_exp, [corr_trials_all[tel_place_cell_index]])
    append!(earlylate_all_exp, [corr_earlylate_all[tel_place_cell_index]])

    
    
end

i_plot = i_job + 1 +already_plot

siginficance_all = []
plot_y_all = []
for i in 1:length(earlylate_median_tog)
    x = reduce(vcat, trials_all_exp[i])
    y = reduce(vcat, earlylate_all_exp[i])

    valid_neuron = isfinite.(x) .* isfinite.(y)
    _,p_value = stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "less")
    siginficance = "n.s."
    if p_value < 0.01
        siginficance = "*"
    end
    if p_value < 1e-3
        siginficance = "**"
    end
    if p_value < 1e-5
        siginficance = "***"
    end
    println(p_value)
    append!(siginficance_all, [siginficance])
        
    
    plot([i_plot*2-1,i_plot*2], [earlylate_median_tog[i], trials_median_tog[i]], color="gray")
    scatter([i_plot*2-1], [earlylate_median_tog[i]], color="gray", alpha=0.7)
    scatter([i_plot*2], [trials_median_tog[i]], color="r", alpha=0.7)
    append!(plot_y_all, trials_median_tog[i])


end
plot_y_all_sort = plot_y_all[sortperm(-plot_y_all)]
siginficance_all_sort = siginficance_all[sortperm(-plot_y_all)] 

for i in 2:length(plot_y_all_sort)
    if plot_y_all_sort[i-1] - plot_y_all_sort[i]  < 0.07
            plot_y_all_sort[i] = plot_y_all_sort[i-1] -  0.07
        end
    end
    
for i in 1:length(earlylate_median_tog) 
    text(i_plot*2 + 0.15, plot_y_all_sort[i], siginficance_all_sort[i])
end

    
    
    append!(tick_locs, [i_plot*2-0.5])
    append!(tick_labels, [])
    already_plot+=1

end

xticks(tick_locs, tick_labels, rotation=90)
ylabel("PF correlation", labelpad=-1)
yticks([-0.3,0, 0.3,0.6])
xlim(0, 23)

fig.savefig(joinpath(fig_path, "PF_correlation_compare_control.pdf"), bbox_inches="tight",transparent = true);

In [ ]:
fig = figure(figsize=(7,1.5))
tick_locs = []
tick_labels = []
i_plot = 1
for (i_job, job_name_original) in enumerate(job_names_ordered)
data_info_all = data_info_dict[job_name_original]
job_name = job_names_figure[i_job]

n_bins = 31

trials_median_tog = []
earlylate_median_tog = []
    
trials_all_exp = []    
earlylate_all_exp = []
    
for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    
    if job_name_original == "boundary_morphing" 
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchangle.h5") 
    elseif job_name_original == "corner_cue_rotation_whole" || job_name_original == "job_retangular_half_circle_rotation" 
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_original.h5")
    else
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchangle.h5")
    end
    file = h5open(info_filename, "r")
    tel_place_cell_index_1 = HDF5.readmmap(file["tel_place_cell_index_1"])
    tel_place_cell_index_2 = HDF5.readmmap(file["tel_place_cell_index_2"])
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    pv_map = HDF5.readmmap(file["pv_map"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_normalized"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    specificity_1 = HDF5.readmmap(file["specificity_1"])
    specificity_2 = HDF5.readmmap(file["specificity_2"])
        
    pv_map_normalized = HDF5.readmmap(file["pv_map_dFF"])
        
    close(file)
        
        
    if info_filename == joinpath(data_path(ds_save_cy_1), "compare_map_results_original.h5")
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late_original.h5")
    else
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late.h5")
    end
    file = h5open(info_filename, "r")
    corr_earlylate_all = HDF5.readmmap(file["corr_earlylate_all"])
    peak_shift_early_late_mm = HDF5.readmmap(file["peak_shift_early_late_mm"])
    specificity_1_early = HDF5.readmmap(file["specificity_1_early"])
    specificity_1_late = HDF5.readmmap(file["specificity_1_late"])
    pv_map_normalized_early_late = HDF5.readmmap(file["pv_map_dFF"])

    confined_place_cell_index_early_late = HDF5.readmmap(file["confined_place_cell_index"])
    close(file)
        
        
    valid_index = isfinite.(pv_map_normalized_early_late) .* isfinite.(pv_map_normalized)
    append!(trials_median_tog, nanmedian(pv_map_normalized[valid_index]))
    append!(earlylate_median_tog, nanmedian(pv_map_normalized_early_late[valid_index]))
    

    append!(trials_all_exp, [pv_map_normalized[valid_index]])
    append!(earlylate_all_exp, [pv_map_normalized_early_late[valid_index]])   

    
    
end



        

siginficance_all = []
plot_y_all = []
for i in 1:length(earlylate_median_tog)
    x = reduce(vcat, trials_all_exp[i])
    y = reduce(vcat, earlylate_all_exp[i])

    valid_neuron = isfinite.(x) .* isfinite.(y)
    _,p_value = stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "less")
    siginficance = "n.s."
    if p_value < 0.01
        siginficance = "*"
    end
    if p_value < 1e-3
        siginficance = "**"
    end
    if p_value < 1e-5
        siginficance = "***"
    end
    println(p_value)
    append!(siginficance_all, [siginficance])
        
    
    plot([i_plot*2-1,i_plot*2], [earlylate_median_tog[i], trials_median_tog[i]], color="gray")
    scatter([i_plot*2-1], [earlylate_median_tog[i]], color="gray", alpha=0.7)
    scatter([i_plot*2], [trials_median_tog[i]], color="r", alpha=0.7)
    append!(plot_y_all, trials_median_tog[i])


end
plot_y_all_sort = plot_y_all[sortperm(-plot_y_all)]
siginficance_all_sort = siginficance_all[sortperm(-plot_y_all)] 

for i in 2:length(plot_y_all_sort)
    if plot_y_all_sort[i-1] - plot_y_all_sort[i]  < 0.07
            plot_y_all_sort[i] = plot_y_all_sort[i-1] -  0.07
        end
    end
    
for i in 1:length(earlylate_median_tog) 
    text(i_plot*2 + 0.15, plot_y_all_sort[i], siginficance_all_sort[i])
end


    
    append!(tick_locs, [i_plot*2-0.5])
    append!(tick_labels, [])
if job_name == "Chamber rotation"
    i_plot +=1
end
    
if job_name == "Wall rotation"
    i_plot +=2
end
    i_plot+=1
    
println(i_plot)
end





already_plot = 0
for (i_job, job_name_original) in enumerate(job_names_ordered)
data_info_all = data_info_dict[job_name_original]
job_name = job_names_figure[i_job]
    
if job_name != "Chamber rotation" && job_name != "Wall rotation"
    continue
end
    

n_bins = 31

trials_median_tog = []
earlylate_median_tog = []
    
trials_all_exp = []    
earlylate_all_exp = []  
    

for which_data in 1:length(data_info_all)
        
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    
    if job_name_original != "boundary_morphing"
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchangle.h5")
    else
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchboundary.h5") 
    end
    file = h5open(info_filename, "r")
    tel_place_cell_index_1 = HDF5.readmmap(file["tel_place_cell_index_1"])
    tel_place_cell_index_2 = HDF5.readmmap(file["tel_place_cell_index_2"])
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])

    pv_map_normalized = HDF5.readmmap(file["pv_map_dFF"])


    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    specificity_1 = HDF5.readmmap(file["specificity_1"])
    specificity_2 = HDF5.readmmap(file["specificity_2"])
    close(file)
        
        
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late.h5")
    file = h5open(info_filename, "r")

    corr_earlylate_all = HDF5.readmmap(file["corr_earlylate_all"])
    corr_pv_earlylate_all = HDF5.readmmap(file["corr_pv_earlylate_all"])
    corr_pv_earlylate_all_normalized = HDF5.readmmap(file["corr_pv_earlylate_all_normalized"])
    peak_shift_early_late_mm = HDF5.readmmap(file["peak_shift_early_late_mm"])
    specificity_1_early = HDF5.readmmap(file["specificity_1_early"])
    specificity_1_late = HDF5.readmmap(file["specificity_1_late"])
    pv_map_normalized_early_late = HDF5.readmmap(file["pv_map_dFF"])
    confined_place_cell_index_early_late = HDF5.readmmap(file["confined_place_cell_index"])
    close(file)
        
        
        
    valid_index = isfinite.(pv_map_normalized_early_late) .* isfinite.(pv_map_normalized)
    append!(trials_median_tog, nanmedian(pv_map_normalized[valid_index]))
    append!(earlylate_median_tog, nanmedian(pv_map_normalized_early_late[valid_index]))
    

    append!(trials_all_exp, [pv_map_normalized[valid_index]])
    append!(earlylate_all_exp, [pv_map_normalized_early_late[valid_index]])   

    
    
end

i_plot = i_job + 1 +already_plot

siginficance_all = []
plot_y_all = []
for i in 1:length(earlylate_median_tog)
    x = reduce(vcat, trials_all_exp[i])
    y = reduce(vcat, earlylate_all_exp[i])

    valid_neuron = isfinite.(x) .* isfinite.(y)
    _,p_value = stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "less")
    siginficance = "n.s."
    if p_value < 0.01
        siginficance = "*"
    end
    if p_value < 1e-3
        siginficance = "**"
    end
    if p_value < 1e-5
        siginficance = "***"
    end
    println(p_value)
    append!(siginficance_all, [siginficance])
        
    
    plot([i_plot*2-1,i_plot*2], [earlylate_median_tog[i], trials_median_tog[i]], color="gray")
    scatter([i_plot*2-1], [earlylate_median_tog[i]], color="gray", alpha=0.7)
    scatter([i_plot*2], [trials_median_tog[i]], color="r", alpha=0.7)
    append!(plot_y_all, trials_median_tog[i])


end
plot_y_all_sort = plot_y_all[sortperm(-plot_y_all)]
siginficance_all_sort = siginficance_all[sortperm(-plot_y_all)] 

for i in 2:length(plot_y_all_sort)
    if plot_y_all_sort[i-1] - plot_y_all_sort[i]  < 0.07
            plot_y_all_sort[i] = plot_y_all_sort[i-1] -  0.07
        end
    end
    
for i in 1:length(earlylate_median_tog) 
    text(i_plot*2 + 0.15, plot_y_all_sort[i], siginficance_all_sort[i])
end

    
    
    append!(tick_locs, [i_plot*2-0.5])
    append!(tick_labels, [])
    already_plot+=1

end

xticks(tick_locs, tick_labels, rotation=90)
ylabel("PV correlation", labelpad=3)

yticks([0, 0.3,0.6])

ylim(-0.1,0.75)

fig.savefig(joinpath(fig_path, "PV_correlation_compare_control.pdf"), bbox_inches="tight",transparent = true);

In [ ]:
job_names_ordered

In [ ]:
fig = figure(figsize=(7,1.5))
tick_locs = []
tick_labels = []
i_plot = 1
for (i_job, job_name_original) in enumerate(job_names_ordered)
data_info_all = data_info_dict[job_name_original]
job_name = job_names_figure[i_job]

n_bins = 31

trials_median_tog = []
earlylate_median_tog = []
    
trials_all_exp = []    
earlylate_all_exp = []  
    

    
for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    
    if job_name_original == "boundary_morphing" 
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchangle.h5") 
    elseif job_name_original == "corner_cue_rotation_whole" || job_name_original == "job_retangular_half_circle_rotation" 
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_original.h5")
    else
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchangle.h5")
    end
        
    file = h5open(info_filename, "r")
    tel_place_cell_index_1 = HDF5.readmmap(file["tel_place_cell_index_1"])
    tel_place_cell_index_2 = HDF5.readmmap(file["tel_place_cell_index_2"])
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    corr_pv_all = HDF5.readmmap(file["corr_pv_all"])
    corr_pv_all_normalized = HDF5.readmmap(file["corr_pv_all_normalized"])
    pv_map = HDF5.readmmap(file["pv_map"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_normalized"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    specificity_1 = HDF5.readmmap(file["specificity_1"])
    specificity_2 = HDF5.readmmap(file["specificity_2"])
    close(file)
        
        
    if info_filename == joinpath(data_path(ds_save_cy_1), "compare_map_results_original.h5")
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late_original.h5")
    else
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late.h5")
    end
    file = h5open(info_filename, "r")
    corr_earlylate_all = HDF5.readmmap(file["corr_earlylate_all"])
    corr_pv_earlylate_all = HDF5.readmmap(file["corr_pv_earlylate_all"])
    corr_pv_earlylate_all_normalized = HDF5.readmmap(file["corr_pv_earlylate_all_normalized"])
    peak_shift_early_late_mm = HDF5.readmmap(file["peak_shift_early_late_mm"])
    specificity_1_early = HDF5.readmmap(file["specificity_1_early"])
    specificity_1_late = HDF5.readmmap(file["specificity_1_late"])
    pv_map_earlylate = HDF5.readmmap(file["pv_map"])
    pv_map_normalized_earlylate = HDF5.readmmap(file["pv_map_normalized"])
    confined_place_cell_index_early_late = HDF5.readmmap(file["confined_place_cell_index"])
    close(file)
        
        
    append!(trials_median_tog, nanmedian(peak_shift_mm[whether_in(confined_place_cell_index,confined_place_cell_index_early_late)]))
    append!(earlylate_median_tog, nanmedian(peak_shift_early_late_mm[whether_in(confined_place_cell_index_early_late, confined_place_cell_index)]))
    
    append!(trials_all_exp, [peak_shift_mm[whether_in(confined_place_cell_index,confined_place_cell_index_early_late)]])
    append!(earlylate_all_exp, [peak_shift_early_late_mm[whether_in(confined_place_cell_index_early_late, confined_place_cell_index)]])   
    

    
    
end

    
println(trials_median_tog)
println(earlylate_median_tog)

        

siginficance_all = []
plot_y_all = []
for i in 1:length(earlylate_median_tog)
    x = reduce(vcat, trials_all_exp[i])
    y = reduce(vcat, earlylate_all_exp[i])

    valid_neuron = isfinite.(x) .* isfinite.(y)
    _,p_value = stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "greater")
    siginficance = "n.s."
    if p_value < 0.01
        siginficance = "*"
    end
    if p_value < 1e-3
        siginficance = "**"
    end
    if p_value < 1e-5
        siginficance = "***"
    end
    append!(siginficance_all, [siginficance])
        
    
    plot([i_plot*2-1,i_plot*2], [earlylate_median_tog[i], trials_median_tog[i]], color="gray")
    scatter([i_plot*2-1], [earlylate_median_tog[i]], color="gray", alpha=0.7)
    scatter([i_plot*2], [trials_median_tog[i]], color="r", alpha=0.7)
    append!(plot_y_all, trials_median_tog[i])


end
plot_y_all_sort = plot_y_all[sortperm(-plot_y_all)]
siginficance_all_sort = siginficance_all[sortperm(-plot_y_all)] 

for i in 2:length(plot_y_all_sort)
    if plot_y_all_sort[i-1] - plot_y_all_sort[i]  < 3
            plot_y_all_sort[i] = plot_y_all_sort[i-1] - 3
        end
    end
    
for i in 1:length(earlylate_median_tog) 
    text(i_plot*2 + 0.15, plot_y_all_sort[i], siginficance_all_sort[i])
end

    
    append!(tick_locs, [i_plot*2-0.5])
    append!(tick_labels, [job_name])
if job_name == "Chamber rotation"
    i_plot +=1
end
    
if job_name == "Wall rotation"
    i_plot +=2
end
    i_plot+=1
    
end






already_plot = 0
for (i_job, job_name_original) in enumerate(job_names_ordered)
data_info_all = data_info_dict[job_name_original]
job_name = job_names_figure[i_job]
    
if job_name != "Chamber rotation" && job_name != "Wall rotation"
    continue
end
    

n_bins = 31

trials_median_tog = []
earlylate_median_tog = []
    
trials_all_exp = []    
earlylate_all_exp = []  
    

for which_data in 1:length(data_info_all)
        
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    
    if job_name_original != "boundary_morphing"
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchangle.h5")
    else
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchboundary.h5") 
    end
    file = h5open(info_filename, "r")
    tel_place_cell_index_1 = HDF5.readmmap(file["tel_place_cell_index_1"])
    tel_place_cell_index_2 = HDF5.readmmap(file["tel_place_cell_index_2"])
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    corr_pv_all = HDF5.readmmap(file["corr_pv_all"])
    corr_pv_all_normalized = HDF5.readmmap(file["corr_pv_all_normalized"])
    pv_map = HDF5.readmmap(file["pv_map"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_normalized"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    specificity_1 = HDF5.readmmap(file["specificity_1"])
    specificity_2 = HDF5.readmmap(file["specificity_2"])
    close(file)
        
        
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late.h5")
    file = h5open(info_filename, "r")

    corr_earlylate_all = HDF5.readmmap(file["corr_earlylate_all"])
    corr_pv_earlylate_all = HDF5.readmmap(file["corr_pv_earlylate_all"])
    corr_pv_earlylate_all_normalized = HDF5.readmmap(file["corr_pv_earlylate_all_normalized"])
    peak_shift_early_late_mm = HDF5.readmmap(file["peak_shift_early_late_mm"])
    specificity_1_early = HDF5.readmmap(file["specificity_1_early"])
    specificity_1_late = HDF5.readmmap(file["specificity_1_late"])
    pv_map_earlylate = HDF5.readmmap(file["pv_map"])
    pv_map_normalized_earlylate = HDF5.readmmap(file["pv_map_normalized"])
    confined_place_cell_index_early_late = HDF5.readmmap(file["confined_place_cell_index"])
    close(file)
        
        
    append!(trials_median_tog, nanmedian(peak_shift_mm[whether_in(confined_place_cell_index,confined_place_cell_index_early_late)]))
    append!(earlylate_median_tog, nanmedian(peak_shift_early_late_mm[whether_in(confined_place_cell_index_early_late, confined_place_cell_index)]))
    
        
    append!(trials_all_exp, [peak_shift_mm[whether_in(confined_place_cell_index,confined_place_cell_index_early_late)]])
    append!(earlylate_all_exp, [peak_shift_early_late_mm[whether_in(confined_place_cell_index_early_late, confined_place_cell_index)]])   
    
    
    
end
    

    
i_plot = i_job + 1 +already_plot

siginficance_all = []
plot_y_all = []
for i in 1:length(earlylate_median_tog)
    x = reduce(vcat, trials_all_exp[i])
    y = reduce(vcat, earlylate_all_exp[i])

    valid_neuron = isfinite.(x) .* isfinite.(y)
    _,p_value = stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "greater")
    siginficance = "n.s."
    if p_value < 0.01
        siginficance = "*"
    end
    if p_value < 1e-3
        siginficance = "**"
    end
    if p_value < 1e-5
        siginficance = "***"
    end
    append!(siginficance_all, [siginficance])
        
    
    plot([i_plot*2-1,i_plot*2], [earlylate_median_tog[i], trials_median_tog[i]], color="gray")
    scatter([i_plot*2-1], [earlylate_median_tog[i]], color="gray", alpha=0.7)
    scatter([i_plot*2], [trials_median_tog[i]], color="r", alpha=0.7)
    append!(plot_y_all, trials_median_tog[i])
        
        



end
plot_y_all_sort = plot_y_all[sortperm(-plot_y_all)]
siginficance_all_sort = siginficance_all[sortperm(-plot_y_all)] 

for i in 2:length(plot_y_all_sort)
    if plot_y_all_sort[i-1] - plot_y_all_sort[i]  < 3
            plot_y_all_sort[i] = plot_y_all_sort[i-1] - 3
        end
    end
    
for i in 1:length(earlylate_median_tog) 
    text(i_plot*2 + 0.15, plot_y_all_sort[i], siginficance_all_sort[i])
end
    
    
    append!(tick_locs, [i_plot*2-0.5])
    append!(tick_labels, [job_name * "\n (rotation corrected)"])
    already_plot+=1

end

xticks(tick_locs, tick_labels, rotation=45, ha="right" , rotation_mode="anchor")
ylabel("PF shift (mm)", labelpad=5)
xlim(0, 23)

ylim(-3,45)
fig.savefig(joinpath(fig_path, "peak_shift_compare_control.pdf"), bbox_inches="tight",transparent = true);